In [4]:
import numpy as np
import pandas as pd

In [5]:
def allocate_portfolio(asset_prices):
    
    # This simple strategy equally weights all assets every period
    # (called a 1/n strategy).
    
    n_assets = len(asset_prices)
    weights = np.repeat(1 / n_assets, n_assets)
    return weights

In [8]:
future_prices = pd.read_csv('Case3HistoricalPrices.csv',index_col=0)[:50]

In [9]:
def compute_sharpe(future_prices):
    '''
    given a dataframe of future prices, compute the sharpe ratio over that interval
    Input:
        - future_prices: a dataframe of future prices
    Output:
        - sharpe ratio over the period covered in future_prices using the allocate_portfolio function
    '''
    #obtain list of time periods
    time_steps = future_prices.index

    #copy the table
    res = future_prices.copy(deep=True)
    res.columns = [asset+' Price' for asset in future_prices.columns]

    #add columns for weights to the table
    temp = pd.DataFrame()
    
    for time_step in time_steps:
        future_price = res.loc[time_step]
        temp = temp.append(pd.Series(allocate_portfolio(future_price)),ignore_index=True)

    temp.columns = [asset+' Weight' for asset in future_prices.columns]
    temp.index = time_steps
    res = pd.merge(res,temp,left_index=True, right_index=True)

    #compute PnL & Value held for each asset
    for asset in future_prices.columns:
        transaction_cost = (res[asset+' Weight']-res[asset+' Weight'].shift()).abs()*res[asset+' Price']*0.003
        res[asset+' PnL'] = (res[asset+' Price']-res[asset+' Price'].shift())*res[asset+' Weight'].shift() - transaction_cost
        res[asset+' Value'] = res[asset+' Weight']*res[asset+' Price']

    #compute Portfolio PnL
    res['Total PnL'] = res[[column for column in res.columns if 'PnL' in column]].sum(axis=1)

    #compute Portfolio Value
    res['Total Value'] = res[[column for column in res.columns if 'Value' in column]].sum(axis=1)

    #compute Returns
    res['Returns'] = res['Total PnL']/res['Total Value'].shift()

    #compute Sharpe
    return res['Returns'].dropna().mean()/res['Returns'].dropna().std()

In [10]:
compute_sharpe(future_prices)

0.024487086434376148